In [295]:
# Import modules
import pandas as pd
import os
import requests
import json
from pandas import json_normalize

In [101]:
# google places api info
gp_secret = os.environ["gp_api"]

In [102]:
gp_resto = pd.read_csv('google_places_poi.csv')
gp_school = pd.read_csv('google_places_school.csv')
gp_park = pd.read_csv('google_places_park.csv')
gp_transit = pd.read_csv('google_places_transit.csv')
gp_grocery = pd.read_csv('google_places_supermarket.csv')


In [103]:
# creating one large dataframe
gp_pois =  pd.concat([gp_resto,gp_school,gp_park,gp_transit,gp_grocery])

In [104]:
gp_pois.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26272 entries, 0 to 2448
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_status                  26108 non-null  object 
 1   icon                             26272 non-null  object 
 2   icon_background_color            26272 non-null  object 
 3   icon_mask_base_uri               26272 non-null  object 
 4   name                             26272 non-null  object 
 5   photos                           17188 non-null  object 
 6   place_id                         26272 non-null  object 
 7   price_level                      5248 non-null   float64
 8   rating                           19053 non-null  float64
 9   reference                        26272 non-null  object 
 10  scope                            26272 non-null  object 
 11  types                            26272 non-null  object 
 12  user_ratings_total 

## We need to run an additional API query on every location in order to get proper address info

In [28]:

def gp_locations (gp_id):
    
    # construct url
    url= 'https://maps.googleapis.com/maps/api/place/details/json?place_id=' + gp_id + '&key=' + gp_secret
    
    # create dictionary for headers
    headers = {}
    #payload = {}
        
    # perform get request
    response = requests.get(url, headers=headers)

    # process request into usable JSON file
    gp_response = json.loads(response.text)
    gp_data.append(gp_response)
    
    return


In [20]:
gp_pois.iloc[0,6]

'ChIJ9aOdYiyNwokRS49czrNTQo4'

In [29]:
gp_data=[]
def gp_api_loop():

    num_rows = gp_pois.shape[0]
    row_count = 0


    while row_count < num_rows:
        gp_locations((gp_pois.iloc[row_count,6]))
        row_count +=1

In [30]:
gp_api_loop()

In [31]:
# saving results as json files for convenience
with open('gp_raw.json', 'w') as f:
    json.dump(gp_data, f)

In [296]:
with open('gp_raw.json') as f:
    gp_data = json.load(f)

In [297]:
# normalizing the json file
gp_raw_df = pd.json_normalize(gp_data)

In [298]:
# investigating results
gp_raw_df.head()

,html_attributions,status,result.address_components,result.adr_address,result.business_status,result.current_opening_hours.open_now,result.current_opening_hours.periods,result.current_opening_hours.weekday_text,result.delivery,result.dine_in,...,result.utc_offset,result.vicinity,result.website,result.wheelchair_accessible_entrance,result.serves_breakfast,result.serves_lunch,result.serves_vegetarian_food,result.curbside_pickup,result.permanently_closed,result.current_opening_hours.special_days
0,[],OK,"[{'long_name': '151', 'short_name': '151', 'ty...","<span class=""street-address"">151 W Sandford Bl...",OPERATIONAL,False,"[{'close': {'date': '2022-11-29', 'day': 2, 't...","[Monday: Closed, Tuesday: 4:00 – 11:00 PM, Wed...",False,True,...,-300,"151 West Sandford Boulevard, Mount Vernon",http://www.riperestaurant.com/,False,NaN,NaN,NaN,NaN,NaN,NaN
1,[],OK,"[{'long_name': '724', 'short_name': '724', 'ty...","<span class=""street-address"">724 Nereid Ave</s...",OPERATIONAL,False,"[{'close': {'date': '2022-12-04', 'day': 0, 't...","[Monday: 11:00 AM – 10:30 PM, Tuesday: 11:00 A...",True,NaN,...,-300,"724 Nereid Avenue, The Bronx",NaN,NaN,False,True,True,NaN,NaN,NaN
2,[],OK,"[{'long_name': '980', 'short_name': '980', 'ty...","<span class=""street-address"">980 E 233rd St</s...",OPERATIONAL,False,"[{'close': {'date': '2022-12-04', 'day': 0, 't...","[Monday: 6:00 AM – 9:00 PM, Tuesday: 6:00 AM –...",True,True,...,-300,"980 East 233rd Street, The Bronx",https://locations.dunkindonuts.com/en/ny/bronx...,True,True,True,NaN,False,NaN,NaN
3,[],OK,"[{'long_name': '980', 'short_name': '980', 'ty...","<span class=""street-address"">980 E 233rd St</s...",OPERATIONAL,False,"[{'close': {'date': '2022-12-04', 'day': 0, 't...","[Monday: 10:00 AM – 10:00 PM, Tuesday: 10:00 A...",True,True,...,-300,"980 East 233rd Street, The Bronx",https://restaurants.subway.com/united-states/n...,True,True,True,True,NaN,NaN,NaN
4,[],OK,"[{'long_name': '1201', 'short_name': '1201', '...","<span class=""street-address"">1201 E 233rd St</...",OPERATIONAL,False,"[{'close': {'date': '2022-12-04', 'day': 0, 't...","[Monday: 10:00 AM – 12:00 AM, Tuesday: 10:00 A...",True,True,...,-300,"1201 East 233rd Street, The Bronx",https://www.popeyes.com/store-locator/store/re...,True,NaN,True,NaN,NaN,NaN,NaN


In [302]:
gp_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26272 entries, 0 to 26271
Data columns (total 56 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   html_attributions                          26272 non-null  object 
 1   status                                     26272 non-null  object 
 2   result.address_components                  26272 non-null  object 
 3   result.adr_address                         26272 non-null  object 
 4   result.business_status                     26108 non-null  object 
 5   result.current_opening_hours.open_now      13839 non-null  object 
 6   result.current_opening_hours.periods       13839 non-null  object 
 7   result.current_opening_hours.weekday_text  13839 non-null  object 
 8   result.delivery                            6985 non-null   object 
 9   result.dine_in                             5140 non-null   object 
 10  result.editorial_summa

In [303]:
# making dataframe of only address data
addresses = gp_raw_df['result.formatted_address'].str.split(',', expand=True)

In [304]:
addresses.head()

,0,1,2,3,4,5,6,7,8
0,151 W Sandford Blvd,Mt Vernon,NY 10550,USA,None,None,None,None,None
1,724 Nereid Ave,The Bronx,NY 10466,USA,None,None,None,None,None
2,980 E 233rd St,The Bronx,NY 10466,USA,None,None,None,None,None
3,980 E 233rd St,The Bronx,NY 10466,USA,None,None,None,None,None
4,1201 E 233rd St,The Bronx,NY 10466,USA,None,None,None,None,None


In [305]:
addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26272 entries, 0 to 26271
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       26272 non-null  object
 1   1       26272 non-null  object
 2   2       26272 non-null  object
 3   3       19855 non-null  object
 4   4       1959 non-null   object
 5   5       113 non-null    object
 6   6       27 non-null     object
 7   7       9 non-null      object
 8   8       3 non-null      object
dtypes: object(9)
memory usage: 1.8+ MB


In [306]:
# dropping unecessary columns
addresses.drop(columns=[3,4,5,6,7,8], inplace=True)

In [307]:
addresses.head()

,0,1,2
0,151 W Sandford Blvd,Mt Vernon,NY 10550
1,724 Nereid Ave,The Bronx,NY 10466
2,980 E 233rd St,The Bronx,NY 10466
3,980 E 233rd St,The Bronx,NY 10466
4,1201 E 233rd St,The Bronx,NY 10466


In [308]:
# splitting state & zip column into two & inserting zip back into frame
addresses[2].str.strip()
state_zip = addresses[2].str.split(' ', expand=True)
addresses[2]= state_zip[2]

In [309]:
# renaming columns
addresses.rename(columns={ 0:'Street Address', 1:'Borough', 2:'Zipcode'},inplace=True)

In [310]:
# not dropping any nulls at this moment in order to maintain dataframe shape
addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26272 entries, 0 to 26271
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Street Address  26272 non-null  object
 1   Borough         26272 non-null  object
 2   Zipcode         19127 non-null  object
dtypes: object(3)
memory usage: 615.9+ KB


In [314]:
addresses.shape[0]

26272

In [327]:
gp_pois.shape[0]

26272

In [312]:
# checking number of unique values pre-merge
print(len(gp_pois['place_id'].unique()))

16842


In [330]:
print(len(addresses['place_id'].unique()))


16842


In [329]:
addresses['place_id'] = gp_pois['place_id']

In [379]:
gp_pois_merged = pd.merge(gp_pois, addresses, on='place_id', how='outer')

In [381]:
gp_pois_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57738 entries, 0 to 57737
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   level_0_x                        57738 non-null  int64  
 1   index_x                          57738 non-null  int64  
 2   business_status                  57178 non-null  object 
 3   icon                             57738 non-null  object 
 4   icon_background_color            57738 non-null  object 
 5   icon_mask_base_uri               57738 non-null  object 
 6   name                             57738 non-null  object 
 7   photos                           42128 non-null  object 
 8   place_id                         57738 non-null  object 
 9   price_level                      9670 non-null   float64
 10  rating                           45643 non-null  float64
 11  reference                        57738 non-null  object 
 12  scope             

In [268]:
gp_pois_merged.head()

,business_status,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,reference,...,plus_code.compound_code,plus_code.global_code,permanently_closed,req_cat,Street Address_x,Borough_x,Zipcode_x,Street Address_y,Borough_y,Zipcode_y
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Ripe Kitchen & Bar,"[{'height': 2471, 'html_attributions': ['<a hr...",ChIJ9aOdYiyNwokRS49czrNTQo4,2.0,4.3,ChIJ9aOdYiyNwokRS49czrNTQo4,...,"V5X6+7F Mount Vernon, NY, USA",87G8V5X6+7F,NaN,restaurant,,,,151 W Sandford Blvd,Mt Vernon,10550
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Ripe Kitchen & Bar,"[{'height': 2471, 'html_attributions': ['<a hr...",ChIJ9aOdYiyNwokRS49czrNTQo4,2.0,4.3,ChIJ9aOdYiyNwokRS49czrNTQo4,...,"V5X6+7F Mount Vernon, NY, USA",87G8V5X6+7F,NaN,restaurant,,,,151 W Sandford Blvd,Mt Vernon,10550
2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Ripe Kitchen & Bar,"[{'height': 2471, 'html_attributions': ['<a hr...",ChIJ9aOdYiyNwokRS49czrNTQo4,2.0,4.3,ChIJ9aOdYiyNwokRS49czrNTQo4,...,"V5X6+7F Mount Vernon, NY, USA",87G8V5X6+7F,NaN,restaurant,,,,151 W Sandford Blvd,Mt Vernon,10550
3,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Ripe Kitchen & Bar,"[{'height': 2471, 'html_attributions': ['<a hr...",ChIJ9aOdYiyNwokRS49czrNTQo4,2.0,4.3,ChIJ9aOdYiyNwokRS49czrNTQo4,...,"V5X6+7F Mount Vernon, NY, USA",87G8V5X6+7F,NaN,school,,,,151 W Sandford Blvd,Mt Vernon,10550
4,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Ripe Kitchen & Bar,"[{'height': 2471, 'html_attributions': ['<a hr...",ChIJ9aOdYiyNwokRS49czrNTQo4,2.0,4.3,ChIJ9aOdYiyNwokRS49czrNTQo4,...,"V5X6+7F Mount Vernon, NY, USA",87G8V5X6+7F,NaN,school,,,,151 W Sandford Blvd,Mt Vernon,10550


In [383]:
# dropping duplicates caused by merge
gp_pois_merged.drop_duplicates(subset=['place_id','name','req_cat'],keep='first', inplace=True)

In [384]:
gp_pois_merged.shape[0]

19164

In [385]:
# making sure number of rows is correct
print(len(gp_pois_merged['place_id'].unique()))

16842


In [386]:
# making sure all categories are still present
gp_pois_merged['req_cat'].unique()

array(['restaurant', 'school', 'park', 'supermarket', 'transit_station'],
      dtype=object)

In [391]:
gp_pois_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19164 entries, 0 to 57737
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   level_0_x                        19164 non-null  int64  
 1   index_x                          19164 non-null  int64  
 2   business_status                  19048 non-null  object 
 3   icon                             19164 non-null  object 
 4   icon_background_color            19164 non-null  object 
 5   icon_mask_base_uri               19164 non-null  object 
 6   name                             19164 non-null  object 
 7   photos                           11841 non-null  object 
 8   place_id                         19164 non-null  object 
 9   price_level                      3622 non-null   float64
 10  rating                           13163 non-null  float64
 11  reference                        19164 non-null  object 
 12  scope             

In [389]:
# exporting as csv for final cleaning
gp_pois_merged.to_csv('gp_pois_merged.csv', index=False)